<a href="https://colab.research.google.com/github/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/03_Deep_Learning_with_Pytorch_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. Deep Learning with Pytorch (sol)

* 파이모션 / Deep RL 핸즈온 [1]
* 김무성

## 차례
* Tensors
* Gradients
* NN building blocks
* Custom layers
* Final glue - loass functions and optimizers
* Monitoring with TensorBoard
* Example - GAN on Atari images

---------------

## Imports and Helper functions

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install torch torchvision > /dev/null 2>&1

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import numpy as np

---------------

# Tensors
* Creation of tensors
* Scalar tensors
* Tensor operations
* GPU tensors


<p align="center"><img src="https://github.com/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/img/cap01.png?raw=1" width=600 /></p>

## Creation of tensors

In [ ]:
a = torch.FloatTensor(3,2)

In [ ]:
a

In [ ]:
a.zero_()

In [ ]:
torch.FloatTensor([[1,2,3],[3,2,1]])

In [ ]:
n = np.zeros(shape=(3, 2))

In [ ]:
n

In [ ]:
n = np.zeros(shape=(3, 2), dtype=np.float32)

In [ ]:
torch.tensor(n)

In [ ]:
n = np.zeros(shape=(3,2))

In [ ]:
torch.tensor(n, dtype=torch.float32)

## Scalar tensors

In [ ]:
a = torch.tensor([1,2,3])

In [ ]:
a

In [ ]:
s = a.sum()

In [ ]:
s

In [ ]:
s.item()

In [ ]:
torch.tensor(1)

## Tensor operations

* https://pytorch.org/docs/stable/index.html

## GPU tensors

In [ ]:
a = torch.FloatTensor([2,3])

In [ ]:
a

In [ ]:
ca = a.cuda()

In [ ]:
ca

In [ ]:
a + 1

In [ ]:
ca + 1

---------------

# Gradients
* Static graph and Dynamic graph
* Tensors and gradients

<p align="center"><img src="https://github.com/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/img/cap02.png?raw=1" width=600 /></p>

### Static graph and Dynamic graph

* [2] https://medium.com/intuitionmachine/pytorch-dynamic-computational-graphs-and-modular-deep-learning-7e7f89f18d1

<img src="https://miro.medium.com/max/2000/1*5PLIVNA5fIqEC8-kZ260KQ.gif" />

---------------

## Tensors and gradients

In [ ]:
v1 = torch.tensor([1.0, 1.0], requires_grad=True)
v1

In [ ]:
v1.grad

In [ ]:
v2 = torch.tensor([2.0, 2.0])
v2

In [ ]:
v_sum = v1 + v2
v_sum

In [ ]:
v_res = (v_sum*2).sum()
v_res

<p align="center"><img src="https://github.com/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/img/cap03.png?raw=1" width=600 /></p>

In [ ]:
v1.is_leaf, v2.is_leaf

In [ ]:
v_sum.is_leaf, v_res.is_leaf

In [ ]:
v1.requires_grad

In [ ]:
v2.requires_grad

In [ ]:
v_sum.requires_grad

In [ ]:
v_res.requires_grad

In [ ]:
v_res.backward()

In [ ]:
v_res

In [ ]:
v_res.grad

In [ ]:
v1.grad

In [ ]:
v1

In [ ]:
v2.grad

---------------

# NN building blocks

Here, we created a randomly initialized feed-forward layer, with two inputs and five outputs, and applied it to our float tensor.

In [ ]:
import torch.nn as nn

In [ ]:
l = nn.Linear(2, 5)

In [ ]:
v = torch.FloatTensor([1, 2])

In [ ]:
l(v)

let's look at useful methods that all nn.Module children provide. They are as follows:
* **parameters()**: A function that returns iterator of all variables which require gradient computation (that is, module weights)
* **zero_grad()**: This function initializes all gradients of all parameters to zero
* **to(device)**: This moves all module parameters to a given device (CPU or GPU)
* **state_dict()**: This returns the dictionary with all module parameters and is useful for model serialization
* **load_state_dict()**: This initializes the module with the state dictionary’

In [ ]:
s = nn.Sequential(
    nn.Linear(2, 5),
    nn.ReLU(),
    nn.Linear(5, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1))

In [ ]:
s

In [ ]:
s(torch.FloatTensor([[1,2]]))

--------------------------

# Custom layers

---------------

In [ ]:
import torch
import torch.nn as nn


In [ ]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.pipe(x)

In [ ]:
net = OurModule(num_inputs=2, num_classes=3)

In [ ]:
print(net)

In [ ]:
v = torch.FloatTensor([[2, 3]])

In [ ]:
out = net(v)

In [ ]:
print(out)

In [ ]:
print("Cuda's availability is %s" % torch.cuda.is_available())

In [ ]:
if torch.cuda.is_available():
        print("Data from cuda: %s" % out.to('cuda'))

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!apt-get install graphviz > /dev/null 2>&1
!pip install graphviz > /dev/null 2>&1
!pip install torchviz > /dev/null 2>&1

In [ ]:
from torchviz import make_dot

In [ ]:
make_dot(net(v), dict(net.named_parameters()))

# Final glue - loass functions and optimizers
* Loss functions
* Optimizers

Now, let's discuss the common blueprint of a training loop:

In [ ]:
for batch_samples, batch_labels in iterate_batches(data, batch_size=32):  # 1
  batch_samples_t = torch.tensor(batch_samples)  # 2 
  batch_labels_t = torch.tensor(batch_labels)    # 3
  out_t = net(batch_samples_t)                   # 4                
  loss_t = loss_function(out_t, batch_labels_t)  # 5
  loss_t.backward()  # 6
  optimizer.step()   # 7
  optimizer.zero_grad()  # 8

---------------

# Monitoring with TensorBoard
* TensorBoard 101
* Plotting stuff

## TensorBoard 101

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install tensorboardX > /dev/null 2>&1

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install -U tensorflow==2.0.0-beta0 > /dev/null 2>&1

In [ ]:
%load_ext tensorboard

<p align="center"><img src="https://github.com/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/img/cap04.png?raw=1" width=600 /></p>

## Plotting stuff

In [ ]:
import math
from tensorboardX import SummaryWriter

In [ ]:
writer = SummaryWriter()

funcs = {"sin": math.sin, "cos": math.cos, "tan": math.tan}

for angle in range(-360, 360):
  angle_rad = angle * math.pi / 180
  for name, fun in funcs.items():
    val = fun(angle_rad)
    writer.add_scalar(name, val, angle)

writer.close()

In [ ]:
!ls runs

In [ ]:
%tensorboard --logdir runs

<p align="center"><img src="https://github.com/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/img/cap05.png?raw=1" width=600 /></p>

---------------

# Example - GAN on Atari images

<p align="center"><img src="https://github.com/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/img/cap06.png?raw=1" width=600 /></p>

<p align="center"><img src="https://github.com/psygrammer/pyemotion_rl/blob/master/examples/ch03_pytorch/img/cap07.png?raw=1" width=600 /></p>

----------------------

# 참고자료
* [1] Deep Reinforcement Learning Hands-On
    - 책 - https://www.amazon.com/dp/B076H9VQH6/
    - github - https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On

* [2] PyTorch, Dynamic Computational Graphs and Modular Deep Learning
  - https://medium.com/intuitionmachine/pytorch-dynamic-computational-graphs-and-modular-deep-learning-7e7f89f18d1